In [7]:
import pickle, os, gc, gzip, json
from io import StringIO
from CGRtools.files import RDFread, SDFread, SDFwrite
from CIMtools.preprocessing import CGR, Fragmentor
from CGRtools.containers import MoleculeContainer

# Split data to small files

In [ ]:
count = 0
with open("/home/aigul/Retro/parsed.rdf", "r") as f:
    i = 0
    for e, line in enumerate(f):
        with open("/home/aigul/Retro/parsed_part_{}.rdf".format(i), "a") as f2:
            f2.write(line)
            if line == "$RXN\n":
                count +=1
                if count % 1000000 == 0:
                    i +=1
                    print(e)

# Create SDF file

In [ ]:
with open("/home/aigul/Retro/Data_for_modeling_100.pickle", "rb") as f:
    rea_centers = pickle.load(f)
reverse={}
for k,v in rea_centers.items():
    for i in v:
        reverse[i]=k

In [ ]:
nums = ["1", "2", "3", "4"]
with SDFwrite("/home/aigul/Retro/training_set.sdf".format, "a") as sdf:
    for i in nums:
        for n,reaction in enumerate(RDFread("/home/aigul/Retro/parsed_part_{}.rdf".format(i))):
            if int(reaction.meta['id']) not in reverse:
                continue
            reaction.meta["rule"]=reverse[int(reaction.meta['id'])]
            for x in reaction.products:
                if x.number_of_nodes()>5:
                    sdf.write(MoleculeContainer(data=x,meta=reaction.meta))

# Create descriptors

In [ ]:
os.environ["PATH"]="/opt/fragmentor"
fr=Fragmentor(version="2017.x", remove_rare_ratio = 0, fragment_type = 3, max_length = 5, min_length = 2) 
#remove_rare_ratio лучше указать !=0, т.к. уменьшить можно, а если не указано, указать другое нельзя
fr.get_params()

In [ ]:
count = 0
part_of_prod = str()
i = 0
with open("/home/aigul/Retro/all_training_set.sdf", "r") as f:
    for line in f:
        part_of_prod += line
        if line == "$$$$\n":
            count +=1
            if count % 10000 == 0:
                mols=SDFread(StringIO(part_of_prod)).read()
                fr.partial_fit(mols)
                del part_of_prod
                del mols
                part_of_prod = str()
                print(count)
                with gzip.open("/home/aigul/Retro/descriptors_"+str(count)+".pickle", "wb", compresslevel=9) as f2:
                    pickle.dump(fr, f2)
    if len(part_of_prod) != 0:
        mols=SDFread(StringIO(part_of_prod)).read()
        fr.partial_fit(mols)
        with gzip.open("/home/aigul/Retro/descriptors_"+str(count)+".pickle", "wb", compresslevel=9) as f2:
            pickle.dump(fr, f2)
with gzip.open("/home/aigul/Retro/descriptors_all.pickle", "wb", compresslevel=9) as f2:
    pickle.dump(fr, f2)

In [ ]:
with gzip.open("/home/aigul/Retro/descriptors_8_3_4/type_8_descriptors/type_8_descriptors_all.pickle","rb") as f3:
    fr = pickle.load(f3)    
fr.remove_rare_ratio = 0.0001
fr.finalize()

In [ ]:
with open("/home/aigul/Retro/descriptors_8_3_4/type_8_descriptors/finalize_descriptors.pickle", "wb") as f4:
    pickle.dump(fr, f4)
os.environ["PATH"]+=":/opt/fragmentor"      #иначе ошибка с fragmentor

In [ ]:
rule_nums_list = []
c = 0
with open("/home/aigul/Retro/all_training_set.sdf","r") as f3:
    for n, line in enumerate(f3):
        if line.find(">  <rule>") != -1:
            c +=1
            num = int(f3.readline())
            rule_nums_list.append(num)
nums_of_rules = sorted(set(rule_nums_list))

In [ ]:
#словарь ключ новые id, значение старые id
count1 = 0
new_old_nums_of_rules = {}
for old_num in nums_of_rules:
    new_old_nums_of_rules[count1] = old_num
    count1 +=1
    
#словарь ключ старые id, значение новые id    
count2 = 0
old_new_nums_of_rules = {}
for old_num in nums_of_rules:
    old_new_nums_of_rules[old_num] = count2 
    count2 +=1
    
with open('/home/aigul/Retro/descriptors_8_3_4/type_8_descriptors/OldNewNumsOfRules.json', 'w') as outfile:
    json.dump(old_new_nums_of_rules, outfile)

In [ ]:
count = 0
old_nums = []
new_nums = []
rea_id = []
year = []
cou = 0
part_of_prod = str()

with open("/home/aigul/Retro/all_training_set.sdf", "r") as f:
    for line in f:
        part_of_prod += line
        if line.find(">  <id>") != -1:
            id_ = int(f.readline())
            rea_id.append(id_)
        if line.find(">  <year>") != -1:
            year_= int(f.readline())
            year.append(year_)
        if line.find(">  <rule>") != -1:
            cou += 1
            num = int(f.readline())
            old_nums.append(num)
            for o_n in old_new_nums_of_rules.keys():
                if o_n == num:
                    new_nums.append(old_new_nums_of_rules[num])
        if line == "$$$$\n":
            count +=1
            if cou % 10000 == 0 and cou != 0:
#             if cou % 2 == 0 and cou != 0:
                mols=SDFread(StringIO(part_of_prod)).read()
                y = fr.transform(mols)
                y["OldNumRule"] = old_nums
                y["NewNumRule"] = new_nums
                y["ReaID"] = rea_id
                y["Year"] = year
                y.to_csv("/home/aigul/Retro/descriptors_8_3_4/type_8_descriptors/transfroms_with_rules/transform_mols_" + str(cou) + ".csv")
                del part_of_prod
                del mols
                part_of_prod = str()
                old_nums = []
                new_nums = []
                year = []
                rea_id = []               
                
#                 break
                
    if len(part_of_prod) != 0:
        mols=SDFread(StringIO(part_of_prod)).read()
        y = fr.transform(mols)
        y["OldNumRule"] = old_nums
        y["NewNumRule"] = new_nums
        y["ReaID"] = rea_id
        y["Year"] = year
        y.to_csv("/home/aigul/Retro/descriptors_8_3_4/type_8_descriptors/transfroms_with_rules/transform_mols_"+str(count)+".csv")    
